<div>lookups</div><div>-----------<br></div>

In [1]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions._
import group.research.aging.spark.extensions._

In [2]:
import ammonite.ops._
import ammonite.ops.ImplicitWd._

<div><b>Making a list of species</b></div><div>-------------------------------<br></div>

In [4]:
val anage = spark.readTSV("/data/databases/anage/anage_data.tsv", headers = true)
 .withColumn("scientific_name", concat($"Genus", lit(" "), $"Species"))
anage.show()

+------+--------+----------+------------+-----------+--------------+-------------+------------+--------------------+----------------------+--------------------+---------------------------+--------------+------------------+-------------------------+--------------------------------+----------------+------------------+----------------+--------------------+-----------------------+------+---------------+-----------+------------+------------+----------+------------------+-------------+---------------+--------------------+--------------------+
|HAGRID| Kingdom|    Phylum|       Class|      Order|        Family|        Genus|     Species|         Common name|Female maturity (days)|Male maturity (days)|Gestation/Incubation (days)|Weaning (days)|Litter/Clutch size|Litters/Clutches per year|Inter-litter/Interbirth interval|Birth weight (g)|Weaning weight (g)|Adult weight (g)|Growth rate (1/days)|Maximum longevity (yrs)|Source|Specimen origin|Sample size|Data quality|IMR (per yr)|MRDT (yrs)|Metabo

In [5]:
anage.columns.toList

List(HAGRID, Kingdom, Phylum, Class, Order, Family, Genus, Species, Common name, Female maturity (days), Male maturity (days), Gestation/Incubation (days), Weaning (days), Litter/Clutch size, Litters/Clutches per year, Inter-litter/Interbirth interval, B

In [6]:
val ensemble_species = spark.readTSV("/data/ensembl/99/Species.csv", sep=",", headers = true)
 .withColumnRenamed("Scientific Name", "scientific_name")
 .withColumnRenamed("Taxon ID", "tax_id")
 .select("scientific_name", "tax_id")

 ensemble_species.show()


+--------------------+-------+
|     scientific_name| tax_id|
+--------------------+-------+
|Orycteropus afer ...|1230840|
|Chelonoidis abing...| 106734|
|Struthio camelus ...| 441894|
|  Gopherus agassizii|  38772|
|         Mus spretus|  10096|
|       Vicugna pacos|  30538|
|Marmota marmota m...|   9994|
|    Poecilia formosa|  48698|
|   Castor canadensis|  51338|
|   Bison bison bison|  43346|
|    Ursus americanus|   9643|
|      Neovison vison| 452646|
|Colobus angolensi...| 336983|
| Anolis carolinensis|  28377|
| Camelus dromedarius|   9838|
| Urocitellus parryii|   9999|
|   Salvator merianae|  96440|
|Dasypus novemcinctus|   9361|
|Scleropages formosus| 113540|
|     Clupea harengus|   7950|
+--------------------+-------+
only showing top 20 rows



In [7]:
val species = anage.join(ensemble_species, "scientific_name")
  .withColumn("url", concat(lit("https://www.ensembl.org/"), $"Genus", lit("_"), $"Species"))
  .withColumnRenamed("tax_id", "taxon_id")
  .sort($"Class", $"Maximum longevity (yrs)".desc)
  .select($"taxon_id".as[String],
      $"scientific_name".as("scientific_name").as[String], 
      $"Common name".as("common_name").as[String],
      $"url".as[String],
  $"Class".as("class").as[String], 
  $"Maximum longevity (yrs)".as("lifespan").as[String],
  $"Body mass (g)".as("mass_g").as[String],
  $"Metabolic rate (W)".as("metabolic_rate"),
  $"Temperature (K)".as("temperature_k"),
  $"Specimen origin".as("specimen_origin").as[String],
  $"Sample size".as("sample_size").as[String],
  $"Data quality".as("data_quality").as[String]
  ).sort($"class", $"lifespan".desc)
species.show()

+--------+--------------------+--------------------+--------------------+-----+--------+------+--------------+-------------+---------------+-----------+------------+
|taxon_id|     scientific_name|         common_name|                 url|class|lifespan|mass_g|metabolic_rate|temperature_k|specimen_origin|sample_size|data_quality|
+--------+--------------------+--------------------+--------------------+-----+--------+------+--------------+-------------+---------------+-----------+------------+
| 2489341| Strigops habroptila|              Kakapo|https://www.ensem...| Aves|    60.0|  null|          null|         null|      captivity|      small|  acceptable|
|  132585|Anser brachyrhynchus|   Pink-footed goose|https://www.ensem...| Aves|    40.9|  null|          null|         null|           wild|     medium|  acceptable|
|    9031|       Gallus gallus|      Red junglefowl|https://www.ensem...| Aves|    30.0|2710.0|         6.005|         null|      captivity|      large|  acceptable|
|   

In [8]:
species.writeParquet("/data/ensembl/99/website/species_lookup.parquet")

parts of /data/ensembl/99/website/species_lookup.parquet merged!


/data/ensembl/99/website/species_lookup.parquet

In [9]:
species.distinct.writeParquet("/data/ensembl/99/website/species_lookup.parquet", true)

parts of /data/ensembl/99/website/species_lookup.parquet merged!


/data/ensembl/99/website/species_lookup.parquet

In [10]:
species.distinct.writeTSV("/data/ensembl/99/website/species_lookup.tsv", true)

parts of /data/ensembl/99/website/species_lookup.tsv merged!


/data/ensembl/99/website/species_lookup.tsv

In [11]:
val prefix_ens = "http://rdf.ebi.ac.uk/resource/ensembl/"
val prefix = "http://aging-research.group/resource/"
def ens(str: String) = "<"+prefix_ens+str.replace(" ", "_")+">"
def tax(str: String) = ens("taxon#"+str)
def pref(str: String) = "<" +prefix +str.replace(" ", "_")+">" 
def str(s: String) = s""""${s.replace("\"", "\\\"")}"""" 
    //+"^^<http://www.w3.org/2001/XMLSchema#string>"

In [12]:
def trip(s: String, p: String, o: String):List[(String, String, String, String)] = if(s==null || p == null || o == null ||
 s == "" ||  p == "" || o == "" || o=="\\N" || o.startsWith("\"\\N\"") || s == "\\N" || s == "\"\\N\"") Nil else List((s,p,o, "<" + "http://rdf.ebi.ac.uk/resource/ensembl/confidence/high" + "> ."))
def sp(taxon_id: java.lang.Integer, scientific_name: String)(p: String, o: Any) = if(taxon_id==null || o == null) Nil else {
      trip(tax(taxon_id.toString), pref(p), o.toString)++
        trip(pref(scientific_name),pref(p), o.toString) 
}

In [13]:
trs.writeTSV("/data/databases/graphdb/import/species.nt", false)

parts of /data/databases/graphdb/import/species.nt merged!


/data/databases/graphdb/import/species.nt

**Making a list of genes**<br>


In [15]:
val gene_members = spark.readTSV("/data/ensembl/99/compara/dump/gene_member.txt").toDF("gene_member_id","stable_id","version","source_name","taxon_id","genome_db_id","biotype_group","canonical_member_id",
"description","dnafrag_id","dnafrag_start","dnafrag_end","dnafrag_strand","display_label").select("stable_id", "display_label", "taxon_id", "biotype_group", "description").distinct()
gene_members.show(10)

+------------------+-------------+--------+-------------+--------------------+
|         stable_id|display_label|taxon_id|biotype_group|         description|
+------------------+-------------+--------+-------------+--------------------+
|ENSPPYG00000017407|           \N|    9601|       coding|                  \N|
|ENSPPYG00000017632|        ELMO1|    9601|       coding|engulfment and ce...|
|ENSPPYG00000017668|           \N|    9601|       coding|                  \N|
|ENSTNIG00000015267|        OTOL1|   99883|       coding|otolin 1 [Source:...|
|ENSTNIG00000007702|        mfsd1|   99883|       coding|major facilitator...|
|ENSPPYG00000018066|         WEE2|    9601|       coding|WEE2 oocyte meios...|
|ENSTNIG00000016189|      zbtb16b|   99883|       coding|zinc finger and B...|
|ENSTNIG00000016255|   zgc:152857|   99883|       coding|zgc:152857 [Sourc...|
|ENSPPYG00000000012|           \N|    9601|       coding|olfactory recepto...|
|ENSPPYG00000000045|         CNST|    9601|       co

In [16]:
val sm_species = species.select("scientific_name", "common_name", "taxon_id", "class")
val genes = gene_members.join(sm_species, "taxon_id").sort($"class", $"scientific_name", $"biotype_group", $"stable_id").distinct()
genes.show()

Error: not found: value species (17)Error: not found: value gene_members (99)

In [17]:
genes.writeParquet("/data/ensembl/99/website/genes_lookup.parquet", true)

parts of /data/ensembl/99/website/genes_lookup.parquet merged!


/data/ensembl/99/website/genes_lookup.parquet

In [18]:
genes.writeTSV("/data/ensembl/99/website/genes_lookup.tsv", true)

parts of /data/ensembl/99/website/genes_lookup.tsv merged!


/data/ensembl/99/website/genes_lookup.tsv

In [19]:
genes.count()

4383816